In [1]:
# K-means (k=3) for the 8 points with initial centroids m1=P1, m2=P4, m3=P7

import numpy as np
import matplotlib.pyplot as plt

# Points
points = np.array([
    [2, 10],  # P1
    [2, 5],   # P2
    [8, 4],   # P3
    [5, 8],   # P4
    [7, 5],   # P5
    [6, 4],   # P6
    [1, 2],   # P7
    [4, 9],   # P8
])
labels = ['P1','P2','P3','P4','P5','P6','P7','P8']

In [2]:
# Initial centroids
m1 = points[0].copy()  # P1
m2 = points[3].copy()  # P4
m3 = points[6].copy()  # P7

def assign_clusters(pts, c1, c2, c3):
    d1 = np.linalg.norm(pts - c1, axis=1)
    d2 = np.linalg.norm(pts - c2, axis=1)
    d3 = np.linalg.norm(pts - c3, axis=1)
    d = np.vstack([d1, d2, d3]).T
    return np.argmin(d, axis=1)   # returns 0,1,2 for nearest centroid

def recompute_centroids(pts, assignments):
    cents = []
    for k in [0,1,2]:
        members = pts[assignments == k]
        if len(members) == 0:
            cents.append(None)
        else:
            cents.append(members.mean(axis=0))
    return cents


In [3]:
# ---------- First assignment (with the provided initial centroids) ----------
assign1 = assign_clusters(points, m1, m2, m3)
members_m1 = [labels[i] for i in range(len(points)) if assign1[i] == 0]
members_m2 = [labels[i] for i in range(len(points)) if assign1[i] == 1]
members_m3 = [labels[i] for i in range(len(points)) if assign1[i] == 2]

print("=== After 1st assignment (initial centroids m1=P1, m2=P4, m3=P7) ===")
print("Members around m1:", members_m1)
print("Members around m2:", members_m2)
print("Members around m3:", members_m3)

centroids_after1 = recompute_centroids(points, assign1)
print("\nCentroids after 1st update:")
print("m1_new =", np.round(centroids_after1[0], 6))
print("m2_new =", np.round(centroids_after1[1], 6))
print("m3_new =", np.round(centroids_after1[2], 6))


=== After 1st assignment (initial centroids m1=P1, m2=P4, m3=P7) ===
Members around m1: ['P1']
Members around m2: ['P3', 'P4', 'P5', 'P6', 'P8']
Members around m3: ['P2', 'P7']

Centroids after 1st update:
m1_new = [ 2. 10.]
m2_new = [6. 6.]
m3_new = [1.5 3.5]


In [4]:
# ---------- Full K-means until convergence ----------
m1_full = m1.copy()
m2_full = m2.copy()
m3_full = m3.copy()

max_iter = 100
tol = 1e-6

for it in range(max_iter):
    assignments = assign_clusters(points, m1_full, m2_full, m3_full)
    new_cents = recompute_centroids(points, assignments)
    # keep previous centroid if cluster becomes empty
    if new_cents[0] is None:
        new_cents[0] = m1_full
    if new_cents[1] is None:
        new_cents[1] = m2_full
    if new_cents[2] is None:
        new_cents[2] = m3_full

    shift = (np.linalg.norm(new_cents[0] - m1_full) +
             np.linalg.norm(new_cents[1] - m2_full) +
             np.linalg.norm(new_cents[2] - m3_full))

    m1_full, m2_full, m3_full = new_cents

    if shift < tol:
        print(f"\nConverged after {it+1} iterations.")
        break
else:
    print(f"\nStopped after max iterations ({max_iter}).")

final_assignments = assignments
final_members_m1 = [labels[i] for i in range(len(points)) if final_assignments[i] == 0]
final_members_m2 = [labels[i] for i in range(len(points)) if final_assignments[i] == 1]
final_members_m3 = [labels[i] for i in range(len(points)) if final_assignments[i] == 2]

print("\n=== Final clustering ===")
print("Cluster around m1 (final):", final_members_m1)
print("Cluster around m2 (final):", final_members_m2)
print("Cluster around m3 (final):", final_members_m3)
print("Final m1 =", np.round(m1_full, 6))
print("Final m2 =", np.round(m2_full, 6))
print("Final m3 =", np.round(m3_full, 6))



Converged after 4 iterations.

=== Final clustering ===
Cluster around m1 (final): ['P1', 'P4', 'P8']
Cluster around m2 (final): ['P3', 'P5', 'P6']
Cluster around m3 (final): ['P2', 'P7']
Final m1 = [3.666667 9.      ]
Final m2 = [7.       4.333333]
Final m3 = [1.5 3.5]


In [5]:
# ---------- Requested answers ----------
index_P6 = labels.index('P6')
cluster_P6 = final_assignments[index_P6]  # 0,1,2
cluster_name = 'm1' if cluster_P6 == 0 else ('m2' if cluster_P6 == 1 else 'm3')
population_m3 = np.sum(final_assignments == 2)

print("\n=== Requested answers (final) ===")
print(f"1) P6 belongs to cluster: {cluster_name} (cluster {cluster_P6})")
print(f"2) Population of cluster around m3: {population_m3}")
print(f"3) Updated centroids:\n   m1 = {np.round(m1_full,6)}\n   m2 = {np.round(m2_full,6)}\n   m3 = {np.round(m3_full,6)}")



=== Requested answers (final) ===
1) P6 belongs to cluster: m2 (cluster 1)
2) Population of cluster around m3: 2
3) Updated centroids:
   m1 = [3.666667 9.      ]
   m2 = [7.       4.333333]
   m3 = [1.5 3.5]
